# Path to dataset

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
path_dataset = '/content/gdrive/My Drive/Doutorado/Bases/GAMMA_training_data/multi-modality_images/'#path to dataset
os.chdir(path_dataset)

Mounted at /content/gdrive


# Setup

In [ ]:
pip install -U tensorflow-addons

In [ ]:
pip install optuna

# 3D CNNs

In [ ]:
!pip install classification-models-3D
!pip install keras_applications
from classification_models_3D.tfkeras import Classifiers

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import glob, warnings
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from collections import Counter#usar para contar a quantidade de uma classe ex: counter(y_train)
warnings.filterwarnings('ignore')
print('TensorFlow Version ' + tf.__version__)

In [ ]:
import zipfile
from io import BytesIO
from keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D, Concatenate, Reshape, GlobalMaxPooling2D, GlobalMaxPooling3D, GlobalAveragePooling3D, Conv2D, Conv1D, Add
from keras import regularizers
from tensorflow.keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
import sklearn
from sklearn.metrics import cohen_kappa_score, roc_auc_score, roc_curve,accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import random
import cv2
import gdown
import matplotlib.pyplot as plt
import requests
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
import random
import sklearn
from sklearn.model_selection import StratifiedKFold, KFold
from tensorflow.keras import layers
import csv
from utils_GAMMA_V2 import padroniza_resultado
from utils_GAMMA_V2 import converte
from scipy import stats
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import DenseNet169
from keras.applications.resnet import ResNet50
from keras.applications.resnet import ResNet101
from keras.applications.resnet import ResNet152
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg19 import VGG19
from keras.models import Sequential, Model
import time
from pathlib import Path
import gc
from keras.backend import int_shape
import pickle
import optuna
from keras.applications.densenet import preprocess_input

# Hyperparameters

In [ ]:
IMAGE_SIZE = 224 #---> Definido no arquivo importado vit.py
depth = 64
# TRAINING
EPOCHS = 150
early_stop_epochs = 2
learning_rate_epochs = 5
optimizer_direction = ['minimize', "maximize"]
number_of_random_points = 25  # random searches to start opt process
#Foldes to save models and dataframes results
teste_number = '1/'
dir_save = '/content/Test_CNN/Models/Test_' + teste_number
results_dir = '/content/Test_CNN/Results_optuna/Test_' + teste_number
classification_results_dir = '/content/Test_CNN/Classification Results Optuna/Test_' + teste_number

# Read Fundus Images and Optic Disc Images (ROIs) - Train

In [ ]:
from utils_GAMMA_V2 import load_fundus_images
folder_fundus = 'dataset_fundus/train/fundus_images flip - 100 images/'
folder_fundus_2 = 'ROI_disco_50px'

conj = 0# 0--> load train images; 1--> load val images
X_train_fundus,X_train_ROI,Y = load_fundus_images(path_dataset,folder_fundus,folder_fundus_2,conj,IMAGE_SIZE)
num_classes = len(np.unique(Y))

100


Read OCTs Volumes

In [ ]:
from utils_GAMMA_V2 import process_scan
from utils_GAMMA_V2 import dir_octs
scan_paths = dir_octs(path_dataset,'multi-modality_images',0)#path to 3D volumes
X_oct = np.array([process_scan(path,IMAGE_SIZE, depth) for path in scan_paths])#read 3D volumes

Split Train/Validation

Fundus

In [ ]:
x_train_fundus, x_val_fundus, y_train, y_val = train_test_split(X_train_fundus, Y, test_size=0.1, random_state=42,stratify=Y)

print(f"x_train shape: {x_train_fundus.shape} - y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val_fundus.shape} - y_val shape: {y_val.shape}")

x_train shape: (90, 224, 224, 3) - y_train shape: (90,)
x_val shape: (10, 224, 224, 3) - y_val shape: (10,)


ROIs

In [ ]:
x_train_ROI, x_val_ROI, y_train, y_val = train_test_split(X_train_ROI, Y, test_size=0.1, random_state=42,stratify=Y)

print(f"x_train shape: {x_train_ROI.shape} - y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val_ROI.shape} - y_val shape: {y_val.shape}")

x_train shape: (90, 224, 224, 3) - y_train shape: (90,)
x_val shape: (10, 224, 224, 3) - y_val shape: (10,)


OCTs

In [ ]:
x_train_oct, x_val_oct, y_train, y_val = train_test_split(X_oct, Y, test_size=0.1, random_state=42,stratify=Y)

print(f"x_train shape: {x_train_oct.shape} - y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val_oct.shape} - y_val shape: {y_val.shape}")

# Read Fundus Images and Optic Disc Images (ROIs) - Test

In [ ]:
from utils_GAMMA_V2 import load_fundus_images
val_path_dataset = '/content/gdrive/My Drive/Doutorado/Bases/GAMMA_val_data/multi-modality_images/'
os.chdir(val_path_dataset)
folder_fundus = 'fundus_images'
folder_fundus_2 = 'ROI_disco_50px'
conj = 1# 1-->Load Images from test set
x_test_fundus,x_test_ROI = load_fundus_images(val_path_dataset,folder_fundus,folder_fundus_2,conj,IMAGE_SIZE)#
print(f"x_teste shape: {x_test_fundus.shape}")

100
x_teste shape: (100, 224, 224, 3)


Read OCT Volumes

In [ ]:
from utils_GAMMA_V2 import process_scan
from utils_GAMMA_V2 import dir_octs

val_path_dataset = '/content/gdrive/My Drive/Doutorado/Bases/GAMMA_val_data/multi-modality_images/'
os.chdir(val_path_dataset)
conj = 1#conj = 0 (Train); conj = 1(Test)
scan_paths = dir_octs(val_path_dataset,'multi-modality_images',conj)
#scan_paths = scan_paths[:1]
X_test_OCT = np.array([process_scan(path,IMAGE_SIZE, depth) for path in scan_paths])#read volumes

# Plot Fundus Images

In [ ]:
 # number of images
n_images = 100

# arrays
x = X_train_fundus
y = Y

# numer of rows and columns
rows = 10
cols = 10

# empty figure
fig, axs = plt.subplots(rows, cols, figsize=(28, 28))
axs = axs.ravel()

# para cada imagem
for i in range(n_images):
    #subplot
    axs[i].imshow(x[i], cmap='gray')
    axs[i].set_title(f'Label: {y[i]}')
    # remove as bordas e o eixo x, y
    axs[i].axis('off')

plt.tight_layout()
plt.show()

# Generate CSV to be evaluated online

In [ ]:
#Gerar o CSV com os resultados obtidos no slit de validação
def gera_csv(predicted_classes, classification_results_dir, trial_number):
  #classification_results_dir = 'content/Test_CNN/Classification Results Optuna/Test_'+teste_number
  if not Path(classification_results_dir).is_dir():
    os.mkdir(classification_results_dir)
  header = ['data','non','early','mid_advanced']
  results = padroniza_resultado(predicted_classes)
  #Alterar a pasta
  results_file_path = os.path.join(val_path_dataset, classification_results_dir  + '/Classification_Results_'+trial_number+'.csv')
  with open(results_file_path, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(results)

# Optimize 2Level Models - V2
Load pre-trained one level models without classifiers.
Freeze some convolutional blocks
New FC Layers are trained

In [ ]:
def objective(trial):#cria e avalia modelo
  num_dense_nodes = trial.suggest_categorical('num_dense_nodes', [64, 128, 256, 512])
  dense_nodes_divisor = trial.suggest_categorical('dense_nodes_divisor',[2, 4, 8])
  batch_size = trial.suggest_categorical('batch_size', [1, 2, 3])
  drop_out_rate=trial.suggest_float('drop_out_rate', 0.0, 0.5, step=0.1)
  freeze_layers = trial.suggest_categorical('freeze_layers',[10, 15, 20, 23])#depends on the number of layers in the model
  lr = trial.suggest_categorical('lr', [0.01, 0.001, 0.0001,])
  num_layers = trial.suggest_categorical('num_layers',[1,2])
  #cnn_model = trial.suggest_categorical('cnn_model', ['InceptionV3','VGG19','resnet50', 'resnet101', 'resnet152', 'Densenet121', 'Densenet169'])
  #CNNs: Dense121, Dense 169, ResNet50, ResNet101, Resnet152, VGG19, Inception V3

  #best model using fundus images as input
  best_model_fundus = keras.models.load_model('Modelos_CNN_Optuna/Testes_50_Trials/Teste_1/22_cnn.h5')#resolution 128x128
  #best model using ROI as input
  best_model_ROI = keras.models.load_model('Modelos_CNN_Optuna/Testes_50_Trials/Teste_2A/16_cnn.h5')#Resolution 128x128

  for layer in best_model_ROI.layers:
    layer._name = layer.name + str("_2")

  best_model_fundus._name = "Level_1"
  best_model_ROI._name = "Level_2"

  #best_model_fundus.trainable = False #Weights will not be updated
  #best_model_ROI.trainable = False #Weights will not be updated

  #freeze convolutions blocks
  for layer in best_model_fundus.layers[:-freeze_layers]:
    layer.trainable= False

  #for layer in best_model_fundus.layers:
    #print(layer.trainable)

  for layer in best_model_ROI.layers[:-freeze_layers]:
    layer.trainable= False

  Level_1 = best_model_fundus.layers[-6].output#Eliminate MLP
  Level_2 = best_model_ROI.layers[-6].output#Eliminate MLP
  #Level_1 = Flatten()(Level_1)
  #Level_2 = Flatten()(Level_2)
  network = Concatenate(axis=-1)([Level_1, Level_2])
  #network = Add()([Level_1, Level_2])
  network = Flatten()(network)
  network = Dense(num_dense_nodes, activation='relu')(network)
  network = Dropout(drop_out_rate)(network)
  if num_layers == 2:
    network = Dense(num_dense_nodes//dense_nodes_divisor, activation='relu')(network)
    network = Dropout(drop_out_rate)(network)
  predictions = Dense(num_classes, activation='softmax')(network)
  model = Model(inputs = [best_model_fundus.input, best_model_ROI.input], outputs = predictions)

  opt = Adam(learning_rate=lr)  # default = 0.001 // learning_rate=lr
  model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy',])#loss='sparse_categorical_crossentropy'
  # callbacks for early stopping and for learning rate reducer
  fn = dir_save + str(trial.number) + '_cnn.hdf5'
  callbacks_list = [EarlyStopping(monitor='val_loss', patience=early_stop_epochs, verbose=1, mode='auto'),
                    #ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=learning_rate_epochs, verbose=0, mode='auto', min_lr=1.0e-6),
                    ModelCheckpoint(filepath=fn, monitor='val_loss',verbose=1,save_weights_only=True,save_best_only=True)
                    ]

  # fit the model
  h = model.fit(x=[x_train_fundus, x_train_ROI], y=y_train,
                          batch_size=batch_size,
                          epochs=EPOCHS,
                          #validation_split=0.1,
                          validation_data = ([x_val_fundus, x_val_ROI],y_val),
                          shuffle=True, verbose=1,
                          callbacks=callbacks_list)


  validation_loss = np.min(h.history['val_loss'])
  val_acc = np.max(h.history['val_accuracy'])
  #Evaluate on test set (define a treshold: val loss or acc )
  prob_class = model.predict(x_test_fundus)#probabilities
  predicted_classes = np.argmax(prob_class, axis=1)
  gera_csv(predicted_classes, classification_results_dir, str(trial.number))
  return validation_loss, val_acc

# Optimize 2Level Models -V1
Models Load with Imagenet Weights

In [ ]:
def objective(trial):#cria e avalia modelo
  num_dense_nodes = trial.suggest_categorical('num_dense_nodes', [64, 128, 256, 512])
  dense_nodes_divisor = trial.suggest_categorical('dense_nodes_divisor',[2, 4, 8])
  batch_size = trial.suggest_categorical('batch_size', [1, 2, 3])
  drop_out_rate=trial.suggest_float('drop_out_rate', 0.0, 0.5, step=0.1)
  lr = trial.suggest_categorical('lr', [0.01, 0.001, 0.0001])
  cnn_model = trial.suggest_categorical('cnn_model', ['InceptionV3','VGG19','resnet50', 'resnet101', 'resnet152', 'Densenet121', 'Densenet169'])
  #CNNs: Dense121, Dense 169, ResNet50, ResNet101, Resnet152, VGG19, Inception V3
  if cnn_model == 'InceptionV3':
    model_fundus = InceptionV3(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

    model_ROI = InceptionV3(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

  if cnn_model == 'VGG19':
    model_fundus = VGG19(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

    model_ROI = VGG19(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

  if cnn_model == 'resnet50':
    model_fundus = ResNet50(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

    model_ROI = ResNet50(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

  if cnn_model == 'resnet101':
    model_fundus = ResNet101(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

    model_ROI = ResNet101(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

  if cnn_model == 'resnet152':
    model_fundus = ResNet152(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

    model_ROI = ResNet152(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

  if cnn_model == 'Densenet121':
    model_fundus = DenseNet121(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

    model_ROI = DenseNet121(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

  if cnn_model == 'Densenet169':
    model_fundus = DenseNet169(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

    model_ROI = DenseNet169(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))


  model_fundus._name = "Level_1"
  model_ROI._name = "Level_2"

  for layer in model_ROI.layers:
      layer._name = layer.name + str("_2")

  Level_1 = model_fundus.output
  Level_2 = model_ROI.output
  #x = Concatenate(axis=-1)([Level_1, Level_2])#feature combination using concatenation
  x = Add()([Level_1, Level_2])#feature combination using addition
  x = Flatten()(x)
  #MLP
  x = Dense(num_dense_nodes, activation='relu')(x)
  x = Dropout(drop_out_rate)(x)
  x = Dense(num_dense_nodes//dense_nodes_divisor, activation='relu')(x)
  x = Dropout(drop_out_rate)(x)
  output_tensor = Dense(num_classes, activation='softmax')(x)
  #Instanciar e compilar modelo
  model = Model(inputs = [model_fundus.input,model_ROI.input], outputs = output_tensor)
  opt = Adam(learning_rate=lr)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])#loss='sparse_categorical_crossentropy'

  # callbacks for early stopping and for learning rate reducer
  #fn = dir_save + str(trial.number) + '_cnn.h5'#save models
  fn = dir_save + str(trial.number) + '_cnn.hdf5'#save only weights
  callbacks_list = [EarlyStopping(monitor='val_loss', patience=early_stop_epochs, verbose=1, mode='auto'),
                    #ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=learning_rate_epochs, verbose=0, mode='auto', min_lr=1.0e-6),
                    ModelCheckpoint(filepath=fn, monitor='val_loss', verbose=1,save_weights_only=True, save_best_only=True)]

  # fit the model
  h = model.fit(x=[x_train_fundus, x_train_ROI], y=y_train,
                          batch_size=batch_size,
                          epochs=EPOCHS,
                          validation_data =([x_val_fundus, x_val_ROI],y_train)
                          #validation_split=0.1,
                          shuffle=True, verbose=1,
                          callbacks=callbacks_list)

  validation_loss = np.min(h.history['val_loss'])
  val_acc = np.max(h.history['val_accuracy'])
  #Evaluate on test set (define a treshold: val loss or acc )
  prob_class = model.predict(x_test_fundus)#probabilities
  predicted_classes = np.argmax(prob_class, axis=1)
  gera_csv(predicted_classes, classification_results_dir, str(trial.number))
  return validation_loss, val_acc

#Optimization

In [ ]:
val_path_dataset = '/content/Test_CNN/'
os.chdir(val_path_dataset)

if not Path(dir_save).is_dir():
  os.mkdir(dir_save)

print('\n*** starting at',pd.Timestamp.now())
start_time_total = time.time()
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(directions=optimizer_direction,study_name="starter-experiment")
study.optimize(objective, n_trials=5,gc_after_trial=True)

In [ ]:
# save results
df_results = study.trials_dataframe()
if not Path(results_dir).is_dir():
  os.mkdir(results_dir)
df_results.to_pickle(results_dir + 'df_optuna_results.pkl')
df_results.to_csv(results_dir + 'df_optuna_results.csv')
elapsed_time_total = (time.time()-start_time_total)/60
print('\n\ntotal elapsed time =',elapsed_time_total,' minutes')



total elapsed time = 9.191481061776479  minutes


# Sort results

In [ ]:
import pandas as pd
val_path_dataset = '/content/Test_CNN/Results_optuna/Test_'+teste_number#path where optuna dataframe was saved
os.chdir(val_path_dataset)
dataframe = pd.read_csv('df_optuna_results.csv')
dataframe.sort_values("values_0",axis=0,ascending=True, inplace=True, na_position='first')
print("Sorted CSV file (according to multiple columns) = ", dataframe.head(10))

Sorted CSV file (according to multiple columns) =     Unnamed: 0  number  values_0  values_1              datetime_start  \
2           2       2  0.924700       0.7  2023-07-30 00:40:38.980133   
3           3       3  1.032561       0.5  2023-07-30 00:43:31.413669   
4           4       4  1.039189       0.5  2023-07-30 00:44:01.420744   
1           1       1  1.049983       0.5  2023-07-30 00:39:52.849290   
0           0       0  3.275835       0.5  2023-07-30 00:38:13.830793   

            datetime_complete                duration  params_batch_size  \
2  2023-07-30 00:43:30.218678  0 days 00:02:51.238545                  1   
3  2023-07-30 00:44:01.053716  0 days 00:00:29.640047                  2   
4  2023-07-30 00:45:18.021475  0 days 00:01:16.600731                  2   
1  2023-07-30 00:40:24.659844  0 days 00:00:31.810554                  1   
0  2023-07-30 00:39:26.512972  0 days 00:01:12.682179                  3   

  params_cnn_model  params_dense_nodes_divisor  param

# Evaluate best models in split test

Test with fundus Images

In [ ]:
best_model = keras.models.load_model('/content/Test_CNN/Models/Test_'+teste_number+'0_cnn.h5')
prob_class = best_model.predict(x_test_fundus)#probabilities
predicted_classes = np.argmax(prob_class, axis=1)
gera_csv(predicted_classes, classification_results_dir)

4/4 [==============================] - 1s 101ms/step


# Ensemble of best models from optimization process

In [ ]:
def make_final_predictions(xprod,
                           models_directory,
                           df_params,
                           threshold, num_models_accept,
                           optimization_direction):

    if optimization_direction == 'maximize':
        df_params.sort_values(by='values_1', ascending=False, inplace=True)
    else:
        df_params.sort_values(by='values_0', ascending=True, inplace=True)

    # apply threshold
    accepted_models_num = 0
    list_predicted_prob = []
    num_models_total = df_params.shape[0]
    for i in range(num_models_total):
        if optimization_direction == 'maximize':
            bool1 = df_params.loc[df_params.index[i],'values_1'] > threshold
        else:
            bool1 = df_params.loc[df_params.index[i],'values_0'] < threshold

        bool2 = df_params.loc[df_params.index[i],'state'] == 'COMPLETE'
        bool3 = accepted_models_num < num_models_accept
        if bool1 and bool2 and bool3:
            model_number = str(df_params.loc[df_params.index[i],'number'])
            #if model_number != '6':
            print (model_number)
            try:
                cnn_model = keras.models.load_model(models_directory + model_number + '_cnn.h5')
            except:
                print('\ncould not read model number:',model_number)
            else:
                list_predicted_prob.append(cnn_model.predict(xprod))
                accepted_models_num = accepted_models_num + 1

    # compute mean probabilities
    mean_probabilities = np.mean(list_predicted_prob, axis=0)

    # compute predicted class
    # argmax uses 1st ocurrance in case of a tie
    y_predicted_class = np.argmax(mean_probabilities, axis=1)
    return y_predicted_class

# fixed parameters - production (Ensemble)
threshold_error =  3# validation loss
number_of_models = 5

# get optuna results parameters
#models_dir = results_directory_stub + 'calibration/'
df_parameters = pd.read_pickle(results_dir + 'df_optuna_results.pkl')

#results_directory = results_directory_stub + calculation_type + '/'
#if not Path(results_directory).is_dir():
    #os.mkdir(results_directory)

final_result = make_final_predictions(x_test_fundus,
                               dir_save,
                               df_parameters,
                               threshold_error,
                               number_of_models, optimizer_direction)

trial_number = 100
gera_csv(final_result, classification_results_dir, str(trial_number))